Copyright (c) 2024 Habana Labs, Ltd. an Intel Company.
SPDX-License-Identifier: Apache-2.0


### Using Hugging Face Pipelines on Intel® Gaudi® 2 - Text Generation

This example shows how to use the Hugging Face Transformers pipeline API to run text generation task on Intel Gaudi.

The pipelines are a great and easy way to use models for inference. These pipelines are objects that abstract most of the complex code from the library, offering a simple API dedicated to several tasks. Text generation pipeline is one of them.

#### Text Generation Pipeline Brief Introduction
Text generation pipeline using any ModelWithLMHead. This pipeline predicts the words that will follow a specified text prompt.

This language generation pipeline can currently be loaded from pipeline() using the following task identifier: "text-generation".

The **models that this pipeline can use are models that have been trained with an autoregressive language modeling objective**, which includes the uni-directional models in the library (e.g. gpt2). See the list of available models on [huggingface.co/models](https://huggingface.co/models?filter=text-generation).

#### Install the Hugging Face Optimum Habana Library

In [ ]:
%cd ~/Gaudi-tutorials/PyTorch/Hugging_Face_pipelines
!pip install optimum-habana==1.15.0

#### Import all neccessary dependencies

In [2]:
import torch
import requests
import torch
from transformers import pipeline

from habana_frameworks.torch.hpu import wrap_in_hpu_graph
import habana_frameworks.torch.core as htcore

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.8/dist-packages/torch/distributed/distributed_c10d.py:252: UserWarning: Device capability of hccl unspecified, assuming `cpu` and `cuda`. Please specify it via the `devices` argument of `register_backend`.
  warnings.warn(


The command below may be needed to modify the existing Hugging Face model classes to use the Intel Gaudi specific version of the model classes.

In [ ]:
from optimum.habana.transformers.modeling_utils import adapt_transformers_to_gaudi
adapt_transformers_to_gaudi()

#### Prepare the input
     We set prompt as a list of 3 prompts, it can do batch inference.

In [3]:
prompts = [
    "Once upon a time, in a land far, far away,",
    "In the beginning, there was darkness.",
    "The quick brown fox jumps over the lazy dog."
]

#### Setup the pipeline
To setup the Hugging Face pipeline we set the following:

* Choose the Hugging Face task: "text-generation"
   This Text generation pipeline can currently be loaded from [`pipeline`] using the following task identifier:
`"text-generation"`.
* Set the device to "hpu" which allows the pipeline to run on Intel Gaudi
* Choose model "gpt2" and data type to be bf16
* Finally we'll use the "wrap_in_hpu_graph" to wrap the module forward function with HPU Graphs. This wrapper captures, caches and replays the graph. More info [here](https://docs.habana.ai/en/latest/PyTorch/Inference_on_PyTorch/Inference_Using_HPU_Graphs.html).

You will see that the Intel Gaudi will build the pipeline

In [4]:
generator = pipeline('text-generation', model = 'gpt2', trust_remote_code=True, torch_dtype=torch.bfloat16, device="hpu")

/usr/local/lib/python3.8/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
============================= HABANA PT BRIDGE CONFIGURATION =========================== 
 PT_HPU_LAZY_MODE = 1
 PT_RECIPE_CACHE_PATH = 
 PT_CACHE_FOLDER_DELETE = 0
 PT_HPU_RECIPE_CACHE_CONFIG = 
 PT_HPU_MAX_COMPOUND_OP_SIZE = 9223372036854775807
 PT_HPU_LAZY_ACC_PAR_MODE = 1
 PT_HPU_ENABLE_REFINE_DYNAMIC_SHAPES = 0
---------------------------: System Configuration :---------------------------
Num CPU Cores : 10
CPU RAM       : 10

In [5]:
generator.model = wrap_in_hpu_graph(generator.model)

#### Execute the pipeline and output the results
Here the input prompts are 3 prompts.
It is batch inference of batch_size =3, and the outputs are list of 3 items.

In [6]:
output = generator(prompts, max_length = 100)

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Extract each output generated_text and display it.

In [7]:
print (f"=== output 0 ===: \n  {output[0][0]['generated_text']} \n")
print (f"=== output 1 ===: \n  {output[1][0]['generated_text']} \n")
print (f"=== output 2 ===: \n {output[2][0]['generated_text']}  \n")

=== output 0 ===: 
  Once upon a time, in a land far, far away, from the Earth, a very beautiful sun had spread forth.

The sun that appeared before him had shone brighter than the sun that had eclipsed him. As the sun had risen from the earth, so had the moon risen from the earth, and the stars were risen from the earth. The earth shook, and the sun struck down upon the city of Sodom, and the stars fell upon Sodom, and the city of 

=== output 1 ===: 
  In the beginning, there was darkness. The night was full of snow, which made the man's mouth water with blood.

Quran: There were many in the city. People were dancing from head to toe. They were in need of a place to hide. The police came to their rescue and helped them. They thought they could kill the man and get away with it. But when the police arrived, they did nothing.

Quran: Allah and his messenger said: " 

=== output 2 ===: 
 The quick brown fox jumps over the lazy dog. 'Have a look,' he says. 'He loves to find new things.' 

In [ ]:
exit()